In [8]:
# The following statement imports the NLTK package.
import nltk

# The following statement imports a class called PlaintextCorpusReader.
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
%matplotlib inline

In [9]:
# load text
filename = 'Data/APPL_transcripts/2020-Jan-29-AAPL.OQ-137948852907-Transcript.txt'
file = open(filename, 'rt')
text = file.read()
file.close()

In [10]:
import nltk
from nltk import sent_tokenize 
tokens = sent_tokenize(text)
print(tokens)

['\n\nRefinitiv StreetEvents Event Transcript\nE D I T E D   V E R S I O N\n\nQ1 2020 Apple Inc Earnings Call\nJANUARY 28, 2020 / 10:00PM GMT\n\n================================================================================\nCorporate Participants\n================================================================================\n\n * Tejas Gala\n   Apple Inc. - IR Contact\n * Luca Maestri\n   Apple Inc. - CFO & Senior VP\n * Timothy D. Cook\n   Apple Inc. - CEO & Director\n\n================================================================================\nConference Call Participiants\n================================================================================\n\n * Krish Sankar\n   Cowen and Company, LLC, Research Division - MD & Senior Research Analyst\n * Christopher Caso\n   Raymond James & Associates, Inc., Research Division - Research Analyst\n * Amit Jawaharlaz Daryanani\n   Evercore ISI Institutional Equities, Research Division - Senior MD & Fundamental Research Analyst\

In [11]:
tokens = [w.lower() for w in tokens]
print(tokens)

['\n\nrefinitiv streetevents event transcript\ne d i t e d   v e r s i o n\n\nq1 2020 apple inc earnings call\njanuary 28, 2020 / 10:00pm gmt\n\n================================================================================\ncorporate participants\n================================================================================\n\n * tejas gala\n   apple inc. - ir contact\n * luca maestri\n   apple inc. - cfo & senior vp\n * timothy d. cook\n   apple inc. - ceo & director\n\n================================================================================\nconference call participiants\n================================================================================\n\n * krish sankar\n   cowen and company, llc, research division - md & senior research analyst\n * christopher caso\n   raymond james & associates, inc., research division - research analyst\n * amit jawaharlaz daryanani\n   evercore isi institutional equities, research division - senior md & fundamental research analyst\

In [12]:
import string
import re 
remove = string.punctuation
remove = re.sub(r"[-]+", "", remove)
pattern = r"[{}]".format(remove + '-')
clean_list = []

for sentence in tokens:
    sentence = re.sub(pattern, "", sentence) 
    sentence = sentence.split()
    sentence = ' '.join(sentence)
    clean_list.append(sentence)

print(clean_list)

# no neeed to remove stopwords and conduct lemmatization
# may worsen prediction for transformer


['refinitiv streetevents event transcript e d i t e d v e r s i o n q1 2020 apple inc earnings call january 28 2020 1000pm gmt corporate participants tejas gala apple inc ir contact luca maestri apple inc cfo senior vp timothy d cook apple inc ceo director conference call participiants krish sankar cowen and company llc research division md senior research analyst christopher caso raymond james associates inc research division research analyst amit jawaharlaz daryanani evercore isi institutional equities research division senior md fundamental research analyst kyle p mcnealy jefferies llc research division equity analyst samik chatterjee jp morgan chase co research division analyst shannon siemsen cross cross research llc cofounder principal analyst wamsi mohan bofa merrill lynch research division director thomas ferris forte da', 'davidson co research division md senior research analyst kathryn lynn huberty morgan stanley research division md and research analyst michael joseph olson 

In [13]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
results = nlp(clean_list)
print(results)

[{'label': 'Neutral', 'score': 0.9999920129776001}, {'label': 'Neutral', 'score': 0.9996411800384521}, {'label': 'Neutral', 'score': 0.9986440539360046}, {'label': 'Neutral', 'score': 0.9998314380645752}, {'label': 'Neutral', 'score': 0.9989511966705322}, {'label': 'Neutral', 'score': 0.9963614344596863}, {'label': 'Neutral', 'score': 0.9635933041572571}, {'label': 'Positive', 'score': 0.9479503631591797}, {'label': 'Neutral', 'score': 0.9999592304229736}, {'label': 'Neutral', 'score': 0.9996371269226074}, {'label': 'Neutral', 'score': 0.9951890707015991}, {'label': 'Neutral', 'score': 0.9736478328704834}, {'label': 'Neutral', 'score': 0.9944841265678406}, {'label': 'Neutral', 'score': 0.9999252557754517}, {'label': 'Neutral', 'score': 0.9986191987991333}, {'label': 'Neutral', 'score': 0.9995105266571045}, {'label': 'Positive', 'score': 0.7907223105430603}, {'label': 'Positive', 'score': 1.0}, {'label': 'Positive', 'score': 1.0}, {'label': 'Positive', 'score': 1.0}, {'label': 'Positive

In [15]:
negative = 0
positive = 0
for element in results:
    if element['label'] == "Negative":
        negative += 1
    elif element['label'] == "Positive":
        positive += 1
total = len(results)

sentiment_score = (positive - negative)/ total
print(sentiment_score)

# +ve sentiment score: positive earnings call
# -ve sentiment score: negative earnings call

# https://www.alpha-sense.com/blog/engineering/sentiment-score/
# sentiment score: (positive - negative) / total number of statements

0.2482598607888631


In [16]:
# # stopword removal
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# words = [w for w in words if not w in stop_words]
# print(words[:100])

In [17]:
# # lemmatization
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# lemmatized = [lemmatizer.lemmatize(word) for word in words]
# print(lemmatized) 